# Data preparation

## Library import

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Flatten
from tensorflow.keras.optimizers import legacy

from sklearn.preprocessing import MinMaxScaler

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.font_manager as font_manager

import time

## Use Colissimo font for Matplotlib

In [2]:
font_path = '/Users/anthony/spyder-py3/Font/Monserrat/Montserrat-Regular.otf'

font_manager.fontManager.addfont(font_path)

plt.rcParams['font.family'] = 'Montserrat'

## Read dataset

In [3]:
# Read main data
data = pd.read_excel('/Users/anthony/spyder-py3/ML/traffic_tg1.xlsx', index_col='date')
data.index = pd.to_datetime(data.index)

# Read holiday day data
holidays_data = pd.read_excel('/Users/anthony/spyder-py3/ML/public_holiday.xlsx')
holidays_data['date'] = pd.to_datetime(holidays_data['date'])

## Create dummies

In [4]:
#Create dummies

# Create a dummy for holiday
data['is_holiday'] = data.index.isin(holidays_data['date']).astype(int)

# Create a dummy for sunday
data['is_sunday'] = (data.index.dayofweek == 6).astype(int)

# Function creation

## Custom MAPE/MAE function loss

In [5]:
def custom_mape(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    mask = tf.math.greater(y_true, 0)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)

    diff = tf.abs((y_true_masked - y_pred_masked) / y_true_masked)
    return 100. * tf.reduce_mean(diff)

def custom_mae(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    mask = tf.math.greater(y_true, 0)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)

    diff = tf.abs(y_true_masked - y_pred_masked)
    return 100. * tf.reduce_mean(diff)

## CNN model

In [6]:
# Define a function to create a sequential CNN model
def create_cnn_model(filters, kernel_size, activation, n_steps_in, n_features, optimizer, loss):
    model = Sequential()
    model.add(Conv1D(filters=filters, 
                     kernel_size=kernel_size, 
                     activation=activation, 
                     input_shape=(n_steps_in, n_features)))
    model.add(Conv1D(filters=filters, 
                     kernel_size=7, 
                     activation=activation))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss=loss, metrics=[custom_mape, custom_mae], run_eagerly=True)
    return model

## Scale data

In [7]:
# Scale data between 0 and 1
def scale_data(data):
    scalers = {}
    data_normalized = data.copy()
    for column in data.columns:
        scaler = MinMaxScaler()
        data_normalized[column] = scaler.fit_transform(data[[column]])
        scalers[column] = scaler
    return data_normalized, scalers

## Data transformation

In [8]:
# Define a function to transfrom data for CNN model
def transform_data_cnn(data_normalized, n_steps_in, start_date, end_date):    
    X, y, dates = [], [], []
    # Filter data based on the start and end dates
    data_filtered = data_normalized[start_date:end_date]
    # Create sequence data of n_steps_in days and n_steps_out days
    for i in range(n_steps_in, len(data_filtered)):
        X.append(data_filtered.iloc[i-n_steps_in:i])
        y.append(data_filtered.iloc[i]['tg1_traffic']) #0 because we just want to predict traffic
        dates.append(data_filtered.index[i])
    X, y, dates = np.array(X), np.array(y), np.array(dates)
    return X, y, dates




## Transfer learning

In [9]:
def transfer_learning(X_train, y_train, n_steps_in, model_params, test_dates, y_test, cnn_model):
    # Initialize empty list to save forecast
    daily_predictions = []
    eight_day_predictions = [0] * len(test_dates)
    
    for i, date in enumerate(test_dates):
        
        # Get traffic of last day to update X_train
        if i == 0:
            new_traffic_point = y_train[-1] # for first forecast, need to take last traffic in train data
        else:
            new_traffic_point = y_test[i-1]
            
        # Create dummies for new datapoint
        current_date = test_dates[i] # forecast's day
        is_holiday = 1 if current_date in holidays_data['date'].values else 0
        is_sunday = 1 if current_date.dayofweek == 6 else 0

        # Create new data point with traffic and dummies
        new_data_point = [new_traffic_point, is_holiday, is_sunday]
        new_data_point_array=np.array([new_data_point])

        # Update X_train and y_train
        new_X = np.append(X_train[-1][1:], new_data_point_array, axis=0)
        X_train = np.append(X_train, [new_X], axis=0)
        y_train = np.append(y_train, [y_test[i]])
    
        # Quick update of model for learning transfer
        print('Training for ',current_date)
        cnn_model.fit(X_train, y_train, epochs=1, verbose=0)

        # Make 1-day forecast
        
        prediction = cnn_model.predict(X_train[-1].reshape(1, n_steps_in, 3)) 
        daily_predictions.append(prediction[0])
        
        # if current date is a Thursday, need to make a 8-day forecast
        if date.weekday() == 3:
            eight_days_data = X_train[-1]
            local_predictions = []
            for k in range(6):
                if i + k < len(test_dates):  # Check that the index does not go beyond the length of test_dates
                    current_date = test_dates[i+k]
                    print(current_date)
                    is_holiday = 1 if current_date in holidays_data['date'].values else 0
                    is_sunday = 1 if current_date.dayofweek == 6 else 0

                    # Créez un nouveau point de données avec le trafic et les dummies
                    new_prediction_point = [prediction[0][0], is_holiday, is_sunday]
                    new_prediction_point_array = np.array([new_prediction_point])
                    eight_days_data = np.append(eight_days_data[1:], new_prediction_point_array, axis=0)
                    prediction = cnn_model.predict(eight_days_data.reshape(1, n_steps_in, 3))
                    local_predictions.append(prediction[0])
            for j in range(min(5, len(local_predictions)-1)):
                index_to_assign = i+2+j
                print(index_to_assign)
                if index_to_assign < len(eight_day_predictions):
                    eight_day_predictions[index_to_assign] = local_predictions[1+j][0]




    return daily_predictions, eight_day_predictions



## Rescale data

In [10]:
def rescale_predictions(predictions, scaler):
    predictions_array = np.array(predictions)
    return scaler.inverse_transform(predictions_array.reshape(-1, 1))

## Plot test results

In [11]:
def plot_test_results_from_df(df, title):
    plt.figure(figsize=(15, 7))
    plt.title(title)
    
    # Extraire les données du DataFrame
    test_dates = df.index
    y_test = df['Trafic TG1'].values
    y_test_forecast = df['Prévision globale'].values
    daily_forecast = df['Prévision journalière'].values
    weekly_forecast = df['Prévision hebdomadaire'].values

    # Remplir la zone d'erreur acceptable (+-5%)
    upper_bound = [val * 1.05 for val in y_test]
    lower_bound = [val * 0.95 for val in y_test]
    plt.fill_between(test_dates, upper_bound, lower_bound, color='blue', alpha=0.2)
    
    # Affichage de la prévision lancée d'un coup
    plt.plot(test_dates, y_test_forecast, label='Prévision globale', color='green', marker='o')
    
    # Affichage des prévisions quotidiennes
    plt.plot(test_dates, daily_forecast, label='Prévision journalière', color='red', marker='x')
    
    # Affichage des prévisions hebdomadaires
    plt.plot(test_dates, weekly_forecast, label='Prévision hebdomadaire', color='purple', marker='s')
    
    # Affichage du trafic réel    
    plt.plot(test_dates, y_test, label='Trafic réel', color='blue', linewidth=2)
    

    plt.xlabel('Date')
    plt.ylabel('Trafic')
    plt.legend()
    plt.grid(False)
    plt.show()

## Plot train results

In [12]:
def plot_train_results_from_df(df, title):
    plt.figure(figsize=(15, 7))
    plt.title(title)
    
    # Extraire les données du DataFrame
    train_dates = df.index
    y_train = df['Trafic TG1'].values
    y_train_forecast = df['Prévision globale'].values

    # Remplir la zone d'erreur acceptable (+-5%)
    upper_bound = [val * 1.05 for val in y_train]
    lower_bound = [val * 0.95 for val in y_train]
    plt.fill_between(train_dates, upper_bound, lower_bound, color='blue', alpha=0.2)
    

    
    
    # Affichage du trafic réel    
    plt.plot(train_dates, y_train, label='Trafic réel', color='blue')
    # Affichage de la prévision lancée d'un coup
    plt.plot(train_dates, y_train_forecast, label='Prévision globale', color='green', linewidth = 2)
    

    plt.xlabel('Date')
    plt.ylabel('Trafic')
    plt.legend()
    plt.grid(False)
    plt.show()

## Calculate MAPE

In [13]:
def calculate_mape(actual, forecast):
    # Éviter la division par zéro et les erreurs NaN
    mask = actual != 0
    return np.mean(np.abs((actual[mask] - forecast[mask]) / actual[mask])) * 100

def compute_mape_for_columns(df, actual_col, forecast_cols):
    mape_values = {}
    for col in forecast_cols:
        valid_forecast_mask = ~df[col].isna()
        mape = calculate_mape(df[actual_col][valid_forecast_mask], df[col][valid_forecast_mask])
        mape_values[col] = mape
    return mape_values

# Data preparation

Normalized the data before creating training and test dataset.
<br>
n_steps_in is the number of week $\times$ number of day in a week (here 7)

In [14]:
data_normalized, scaler = scale_data(data)

n_steps_in =365

X_train, y_train, train_dates = transform_data_cnn(data_normalized, 
                                                   n_steps_in,  
                                                   '2017-01-01', 
                                                   '2021-12-31')

X_test, y_test, test_dates = transform_data_cnn(data_normalized, 
                                                n_steps_in, 
                                                '2022-01-01', 
                                                '2023-01-15')

# Model creation

## Model parameters

In [15]:
# Paramètres du modèle
model_params = {
    "filters": 32,
    "kernel_size": 7,
    "activation": "relu",
    "n_steps_in": n_steps_in,
    "n_features": 3,
    "optimizer": 'adam',
    "loss": custom_mape
}

## Model creation

In [16]:
cnn_model = create_cnn_model(**model_params)
cnn_model.fit(X_train, y_train, epochs=20)

2023-08-16 15:02:16.386481: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-08-16 15:02:16.386549: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-08-16 15:02:16.386559: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-08-16 15:02:16.387106: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-16 15:02:16.387606: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/20
46/46 [==============================] - 2s 35ms/step - loss: 87.9533 - custom_mape: 87.4961 - custom_mae: 16.6008
Epoch 2/20
46/46 [==============================] - 1s 30ms/step - loss: 70.3446 - custom_mape: 70.7101 - custom_mae: 12.8356
Epoch 3/20
46/46 [==============================] - 1s 31ms/step - loss: 46.2149 - custom_mape: 46.1002 - custom_mae: 11.0208
Epoch 4/20
46/46 [==============================] - 1s 32ms/step - loss: 43.2987 - custom_mape: 43.3342 - custom_mae: 9.9039
Epoch 5/20
46/46 [==============================] - 2s 35ms/step - loss: 47.9973 - custom_mape: 48.2189 - custom_mae: 11.9250
Epoch 6/20
46/46 [==============================] - 2s 34ms/step - loss: 41.7451 - custom_mape: 41.6130 - custom_mae: 10.2363
Epoch 7/20
46/46 [==============================] - 2s 33ms/step - loss: 42.0227 - custom_mape: 42.0473 - custom_mae: 10.2329
Epoch 8/20
46/46 [==============================] - 2s 38ms/step - loss: 44.2211 - custom_mape: 44.0048 - custom_mae: 8

## Global forecast

In [17]:
y_train_forecast = cnn_model.predict(X_train)
y_test_forecast = cnn_model.predict(X_test)

1/1 [==============================] - 0s 20ms/step


## Daily and Weekly Forecast

In [ ]:
start_time = time.time()
daily_forecast, weekly_forecast = transfer_learning(X_train,y_train,n_steps_in,model_params,test_dates,y_test,cnn_model)
print("--- Transfert d'apprentissage: %s seconds ---" % (time.time() - start_time))

Training for  2023-01-01 00:00:00
1/1 [==============================] - 0s 66ms/step
Training for  2023-01-02 00:00:00
1/1 [==============================] - 0s 10ms/step
Training for  2023-01-03 00:00:00
1/1 [==============================] - 0s 10ms/step
Training for  2023-01-04 00:00:00
1/1 [==============================] - 0s 10ms/step
Training for  2023-01-05 00:00:00
1/1 [==============================] - 0s 17ms/step
2023-01-05 00:00:00
1/1 [==============================] - 0s 11ms/step
2023-01-06 00:00:00
1/1 [==============================] - 0s 11ms/step
2023-01-07 00:00:00
1/1 [==============================] - 0s 10ms/step
2023-01-08 00:00:00
1/1 [==============================] - 0s 10ms/step
2023-01-09 00:00:00
1/1 [==============================] - 0s 10ms/step
2023-01-10 00:00:00
1/1 [==============================] - 0s 10ms/step
6
7
8
9
10
Training for  2023-01-06 00:00:00
1/1 [==============================] - 0s 10ms/step
Training for  2023-01-07 00:00:00
1/1 [==

# Data analysis

## Rescale data

In [ ]:
y_train_forecast_rescaled = rescale_predictions(y_train_forecast, scaler['tg1_traffic'])
y_train_rescaled = rescale_predictions(y_train, scaler['tg1_traffic'])


y_test_forecast_rescaled = rescale_predictions(y_test_forecast, scaler['tg1_traffic'])
y_test_rescaled = rescale_predictions(y_test, scaler['tg1_traffic'])



daily_forecast_rescaled = rescale_predictions(daily_forecast, scaler['tg1_traffic'])
weekly_forecast_rescaled = rescale_predictions(weekly_forecast, scaler['tg1_traffic'])

## Concatenate forecast in dataframe

In [ ]:
data_train_forecast = pd.DataFrame({
    'Trafic TG1' : y_train_rescaled.flatten().tolist() ,
    'Prévision globale' : y_train_forecast_rescaled.flatten().tolist()
                            }, index = train_dates.tolist())

data_test_forecast = pd.DataFrame({
    'Trafic TG1' : y_test_rescaled.flatten().tolist() ,
    'Prévision globale' : y_test_forecast_rescaled.flatten().tolist(),
    'Prévision hebdomadaire' : weekly_forecast_rescaled.flatten().tolist(),
    'Prévision journalière' : daily_forecast_rescaled.flatten().tolist()  
                            }, index = test_dates.tolist())

## Plot results

In [ ]:
plot_test_results_from_df(data_test_forecast, 'Transfert d’apprentissage')

In [ ]:
plot_train_results_from_df(data_train_forecast, "Résultats d'entraînement")

## Metrics

In [ ]:
# Utilisation de la fonction:
forecast_columns = ['Prévision globale', 'Prévision journalière', 'Prévision hebdomadaire']
mapes = compute_mape_for_columns(data_test_forecast, 'Trafic TG1', forecast_columns)

for col, mape in mapes.items():
    print(f"MAPE {col} : {mape:.2f}%")

In [ ]:
# Utilisation de la fonction:
forecast_columns = ['Prévision globale']
mapes = compute_mape_for_columns(data_train_forecast, 'Trafic TG1', forecast_columns)

for col, mape in mapes.items():
    print(f"MAPE {col} : {mape:.2f}%")